In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
import matplotlib.patches as mpatches
plt.rcParams['figure.dpi'] = 120
from PAM import *
from CLARA import *
from scipy import stats
from sklearn.preprocessing import StandardScaler

def descriptive_statistics(df):
  kurtosis = df.kurtosis(axis=0)+3 # +3 because default is excess kurtosis
  skewness = df.skew(axis=0)
  description = df.describe()
  description.loc['kurtosis'] = kurtosis
  description.loc['skewness'] = skewness
  display(description.apply(lambda s: s.apply('{0:.4f}'.format)))
  return description

### Start Pre-processing

In [ ]:
path_to_file='./OnlineRetail.csv'
df=pd.read_csv(path_to_file,encoding='latin1').dropna() ## had NaN data in some of the customer Ids
df.head(5)

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%d-%m-%Y %H:%M') ## to datetime
df.drop(['StockCode','Description','Country'],axis = 1, inplace =True) ## unecessary columns
df['CustomerID']=df['CustomerID'].astype(int)

In [ ]:
df.info()

In [ ]:
df_descript=descriptive_statistics(df[['UnitPrice','Quantity']])

In [ ]:
df[df['CustomerID'].isna()]

In [ ]:
df=df[(df['UnitPrice']>0) & (df['Quantity']>0)] #remove negative pricing and quantity which is clearly wrong
df_descript=descriptive_statistics(df[['UnitPrice','Quantity']])

In [ ]:
import plotly.express as px

# Assuming df is your DataFrame
top_customers = df['CustomerID'].value_counts().nlargest(5)

# Convert CustomerID to string to ensure categorical plotting
fig = px.bar(top_customers, x=top_customers.index.astype(str), y=top_customers.values,
             labels={'x': 'Customer ID', 'y': 'Number of Orders'},
             title='Top 5 Customers by Number of Orders',
             width=800,height=800,color_discrete_sequence=['#4eba67'])

# Customize the layout for better readability
fig.update_layout(xaxis_type='category')
fig.show()


In [ ]:
# which month is more "active"
# Create a month column
df.loc[:,'Month'] = df['InvoiceDate'].copy().dt.month

# Count orders per month
orders_per_month = df['Month'].value_counts().sort_index()

# Create the bar chart
fig = px.bar(orders_per_month, x=orders_per_month.index, y=orders_per_month.values,
             labels={'x': 'Month', 'y': 'Number of Orders'},
             title='Monthly Order Activity',
             width=800, height=800,color_discrete_sequence=['#4eba67'])
fig.show()

In [ ]:
### RFM features ###

import pandas as pd
from datetime import datetime

# Monetary
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# The most recent date in the dataset
snapshot_date = df['InvoiceDate'].max() + pd.DateOffset(days=1)

# Aggregate data by each customer
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency
    'TotalPrice': 'sum'  # Monetary Value
})

# Rename columns
rfm.rename(columns={'InvoiceDate': 'Recency',
                    'InvoiceNo': 'Frequency',
                    'TotalPrice': 'MonetaryValue'}, inplace=True)

# Check the first few rows of the data
display(rfm.head())


In [ ]:
rfm_descript=descriptive_statistics(rfm)

In [ ]:
# Histogram for Recency
fig = px.histogram(rfm, x='Recency',
                   title='Distribution of Recency',
                   labels={'Recency': 'Recency (Days)'},
                   nbins=30,  # Adjust the number of bins as needed
                   width=800, height=800,color_discrete_sequence=['#4eba67'] )
fig.update_layout(bargap=0.2)
fig.show()

# Histogram for Frequency
fig = px.histogram(rfm, x='Frequency',
                   title='Distribution of Frequency',
                   labels={'Frequency': 'Frequency (Number of Purchases)'},
                   nbins=30,  # Adjust the number of bins as needed
                   width=800, height=800,color_discrete_sequence=['#4eba67'])
fig.update_layout(bargap=0.2)
fig.show()

# Histogram for Monetary Value
fig = px.histogram(rfm, x='MonetaryValue',
                   title='Distribution of Monetary Value',
                   labels={'MonetaryValue': 'Monetary Value (Total Spend)'},
                   nbins=30,  # Adjust the number of bins as needed
                   width=800, height=800,color_discrete_sequence=['#4eba67'])
fig.update_layout(bargap=0.2)
fig.show()



In [ ]:
# Violin plot for Recency
fig = px.violin(rfm, y='Recency',
                box=True,  # shows box plot inside the violin
                points='all',  # shows all points
                title='Violin Plot of Recency',
                labels={'Recency': 'Recency (Days)'},
                width=800, height=800,color_discrete_sequence=['#4eba67']
               )
fig.update_traces(box_visible=True, box_fillcolor='#cf8a13')

fig.show()

# Violin plot for Frequency
fig = px.violin(rfm, y='Frequency',
                box=True,
                points='all',
                title='Violin Plot of Frequency',
                labels={'Frequency': 'Frequency (Number of Purchases)'},
                width=800, height=800,color_discrete_sequence=['#4eba67']
               )
fig.update_traces(box_visible=True, box_fillcolor='#cf8a13')

fig.show()

# Violin plot for Monetary Value
fig = px.violin(rfm, y='MonetaryValue',
                box=True,
                points='all',
                title='Violin Plot of Monetary Value',
                labels={'MonetaryValue': 'Monetary Value (Total Spend)'},
                width=800, height=800,color_discrete_sequence=['#4eba67']
               )
fig.update_traces(box_visible=True, box_fillcolor='#cf8a13')

fig.show()



#### Data scaling for clustering

In [ ]:
# Assuming 'rfm' is your DataFrame
scaler = StandardScaler()

# Fit and transform the data
rfm_scaled = scaler.fit_transform(rfm)

# Convert the scaled data back to a DataFrame
rfm_scaled_df = pd.DataFrame(rfm_scaled, index=rfm.index, columns=rfm.columns)

# Display the first few rows of the scaled data
display(rfm_scaled_df.head())

In [ ]:
# Violin plot for Recency
fig = px.violin(rfm_scaled_df, y='Recency',
                box=True,  # shows box plot inside the violin
                points='all',  # shows all points
                title='Violin Plot of Recency',
                labels={'Recency': 'Recency (Days)'},
                width=800, height=800,color_discrete_sequence=['#4eba67']
               )
fig.update_traces(box_visible=True, box_fillcolor='#cf8a13')

fig.show()

# Violin plot for Frequency
fig = px.violin(rfm_scaled_df, y='Frequency',
                box=True,
                points='all',
                title='Violin Plot of Frequency',
                labels={'Frequency': 'Frequency (Number of Purchases)'},
                width=800, height=800,color_discrete_sequence=['#4eba67']
               )
fig.update_traces(box_visible=True, box_fillcolor='#cf8a13')

fig.show()

# Violin plot for Monetary Value
fig = px.violin(rfm_scaled_df, y='MonetaryValue',
                box=True,
                points='all',
                title='Violin Plot of Monetary Value',
                labels={'MonetaryValue': 'Monetary Value (Total Spend)'},
                width=800, height=800,color_discrete_sequence=['#4eba67']
               )
fig.update_traces(box_visible=True, box_fillcolor='#cf8a13')

fig.show()



### Apply clustering

In [ ]:
# Store the results
results = {'Number of Clusters':[],
           'Average Silhouette':[],
           'Average MSD':[]}

for k in range(2,8+1):
    # Instantiate and fit the clustering model
    # Replace 'CLARA' with 'PAM' if you want to use PAM instead
    print(f'Testing {k} clusters...')
    clustering_model = CLARA(rfm_scaled_df, k, sample_size=200, num_samples=10)
    clustering_model.fit(verbose=0)

    # Evaluate the clustering
    silhouette_avg = clustering_model.sample_silhouette_avg
    msd_avg = clustering_model.sample_msd_avg
    # Store the results
    results['Number of Clusters'].append(k)
    results['Average Silhouette'].append(silhouette_avg)
    results['Average MSD'].append(msd_avg)

    clustering_model.generate_clara_report(file_name=f'Onlineshop_{k}clusters_CLARA_Report.md',save_markdown=True,path='./Project')

results_df=pd.DataFrame(results).set_index('Number of Clusters')
results_df

In [ ]:
      ###elbow method
results_df.plot(y='Average Silhouette')

In [ ]:
###elbow method

results_df.plot(y='Average MSD')

In [ ]:
k=3
 #best choice based on metrics
clustering_model = CLARA(rfm_scaled_df, k, sample_size=300, num_samples=10)
clustering_model.fit(verbose=0)
clustering_model.generate_clara_report(file_name=f'Onlineshop_final_{k}clusters_CLARA_Report.md',save_markdown=True,path='./Project')

In [ ]:
clustering_model.plot_silhouette(sample_size=10000)